# Data

In [ ]:
from tqdm import tqdm as log_progress

from itertools import chain
from razdel.utils import (
    join_path,
    list_paths,
    load_lines,
    dump_lines,
    load_xml
)
from razdel.eval.etl.syntag import (
    parse_sents as parse_syntag_sents,
    parse_tokens as parse_syntag_tokens
)
from razdel.eval.etl.corpora import (
    parse_sents as parse_corpora_sents,
    parse_tokens as parse_corpora_tokens
)
from razdel.eval.etl.gicrya import (
    parse_sents as parse_gicrya_sents,
    parse_tokens as parse_gicrya_tokens
)
from razdel.eval.etl.rnc import (
    parse_sents as parse_rnc_sents,
    parse_tokens as parse_rnc_tokens
)
from razdel.tests.partition import (
    format_partitions,
    parse_partitions
)

## Syntag

In [ ]:
# !mkdir -p data/raw

In [ ]:
# !wget https://github.com/UniversalDependencies/UD_Russian-SynTagRus/archive/master.zip -P data/raw
# !unzip data/raw/master.zip -d data/raw
# !rm data/raw/master.zip

In [ ]:
# paths = list_paths(join_path('data', 'raw', 'UD_Russian-SynTagRus-master', '*.conllu'))
# lines = chain.from_iterable(load_lines(_) for _ in paths)
# partitions = parse_syntag_sents(lines)
# partitions = log_progress(partitions)
# lines = format_partitions(partitions)
# dump_lines(lines, join_path('data', 'syntag_sents.txt'))

In [ ]:
# paths = list_paths(join_path('data', 'raw', 'UD_Russian-SynTagRus-master', '*.conllu'))
# lines = chain.from_iterable(load_lines(_) for _ in paths)
# partitions = parse_syntag_tokens(lines)
# partitions = log_progress(partitions)
# lines = format_partitions(partitions)
# dump_lines(lines, join_path('data', 'syntag_tokens.txt'))

## Corpora

In [ ]:
# !wget http://opencorpora.org/files/export/annot/annot.opcorpora.xml.bz2 -P data/raw
# !bunzip2 -d data/raw/annot.opcorpora.xml.bz2

In [ ]:
# stream = load_xml(join_path('data', 'raw', 'annot.opcorpora.xml'))
# records = parse_corpora_sents(stream)
# records = log_progress(records)
# lines = format_partitions(partitions)
# dump_lines(lines, join_path('data', 'corpora_sents.txt'))

In [ ]:
# stream = load_xml(join_path('data', 'raw', 'annot.opcorpora.xml'))
# partitions = parse_corpora_tokens(stream)
# partitions = log_progress(partitions)
# lines = format_partitions(partitions)
# dump_lines(lines, join_path('data', 'corpora_tokens.txt'))

## Gicrya

In [ ]:
# !wget https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/GICRYA_texts.zip -P data/raw
# !unzip data/raw/GICRYA_texts.zip -d data/raw
# !rm data/raw/GICRYA_texts.zip

In [ ]:
# lines = load_lines(join_path('data', 'raw', 'gikrya_fixed.txt'))
# partitions = parse_gicrya_sents(lines)
# partitions = log_progress(partitions)
# lines = format_partitions(partitions)
# dump_lines(lines, join_path('data', 'gicrya_sents.txt'))

In [ ]:
# lines = load_lines(join_path('data', 'raw', 'gikrya_fixed.txt'))
# partitions = parse_gicrya_tokens(lines)
# partitions = log_progress(partitions)
# lines = format_partitions(partitions)
# dump_lines(lines, join_path('data', 'gicrya_tokens.txt'))

## RNC

In [ ]:
# !wget https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/RNC_texts.rar -P data/raw
# !unrar x data/raw/RNC_texts.rar 
# !mv RNCgoldInUD_Morpho.conll data/raw
# !rm data/raw/RNC_texts.rar

In [ ]:
# lines = load_lines(join_path('data', 'raw', 'RNCgoldInUD_Morpho.conll'))
# partitions = parse_rnc_sents(lines)
# partitions = log_progress(partitions)
# lines = format_partitions(partitions)
# dump_lines(lines, join_path('data', 'rnc_sents.txt'))

In [ ]:
# lines = load_lines(join_path('data', 'raw', 'RNCgoldInUD_Morpho.conll'))
# partitions = parse_rnc_tokens(lines)
# partitions = log_progress(partitions)
# lines = format_partitions(partitions)
# dump_lines(lines, join_path('data', 'rnc_tokens.txt'))

# Sent

In [ ]:
from razdel.utils import (
    join_path,
    load_lines,
)
from razdel.tests.partition import parse_partitions


lines = load_lines(join_path('data', 'syntag_sents.txt'))
syntag_sents = list(parse_partitions(lines))

lines = load_lines(join_path('data', 'corpora_sents.txt'))
corpora_sents = list(parse_partitions(lines))

lines = load_lines(join_path('data', 'gicrya_sents.txt'))
gicrya_sents = list(parse_partitions(lines))

lines = load_lines(join_path('data', 'rnc_sents.txt'))
rnc_sents = list(parse_partitions(lines))

In [ ]:
from razdel.eval.zoo import dot_sentenize

# pip install rusenttokenize
from razdel.eval.zoo import deepmipt_sentenize

# pip install nltk
# nltk.download('punkt')
# wget https://raw.githubusercontent.com/mhq/train_punkt/master/russian.pickle
#      -O ~/nltk_data/tokenizers/punkt/PY3/russian.pickle
from razdel.eval.zoo import nltk_sentenize

# pip install segtok
from razdel.eval.zoo import segtok_sentenize

from razdel import sentenize


# Texterra
# Можно ещё сравнивать с https://texterra.ispras.ru/products, но
# 1. она медленно работает, как минимум затраты на http
# 2. иногда кидает ошибку (возможно дело в английских предложениях)
# 3. качество немного выше segtok


# Polyglot
# реализует http://www.unicode.org/reports/tr29/

# Сорян, не смог установить. Дикие траблы с ICU
# brew install icu4c
# export ICU_VERSION=62.1
# export BASE=/usr/local/Cellar/icu4c/
# export PATH=$PATH:$BASE/$ICU_VERSION/bin
# export PYICU_INCLUDES=$BASE/$ICU_VERSION/include
# export PYICU_LFLAGS=-L$BASE/$ICU_VERSION/lib
# pip install pyicu polyglot

# Вроде установилось но 
# > from polyglot.text import Text
# > Text('...')
# Symbol not found: __ZNK6icu_6214Transliterator12getTargetSetERNS_10UnicodeSetE

In [ ]:
%%time
from tqdm import tqdm as log_progress

from joblib import Parallel, delayed

from razdel.eval.tests import (
    generate_precision_tests,
    generate_recall_tests,
    correct_precision,
    correct_recall
)
from razdel.eval.report import (
    Dataset,
    Model,
    Metric,
    report_tasks,
    run_task
)

datasets = [
    Dataset('corpora', corpora_sents),
    Dataset('syntag', syntag_sents),
    Dataset('gicrya', gicrya_sents),
    Dataset('rnc', rnc_sents),
]
models = [
    Model('re.split([.?!…])', dot_sentenize),
    Model('nltk.sent_tokenize', nltk_sentenize),
    Model('segtok.split_single', segtok_sentenize),
    Model('deepmipt', deepmipt_sentenize),
    Model('razdel.sentenize', sentenize),
]
metrics = [
    Metric('precision', generate_precision_tests, correct_precision),
    Metric('recall', generate_recall_tests, correct_recall)
]


tasks = list(report_tasks(datasets, models, metrics))
results = Parallel(n_jobs=4)(
    delayed(run_task)(_)
    for _ in log_progress(tasks)
)

In [ ]:
import pandas as pd

from razdel.eval.report import (
    result_rows,
    show_results
)


table = pd.DataFrame(result_rows(results))
print(show_results(table).to_html())
show_results(table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>corpora</th>
      <th>syntag</th>
      <th>gicrya</th>
      <th>rnc</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>re.split([.?!…])</th>
      <td>8027 (5615 + 2412), 13.1s</td>
      <td>2188 (1761 + 427), 6.1s</td>
      <td>4096 (2413 + 1683), 8.3s</td>
      <td>8191 (6587 + 1604), 9.4s</td>
    </tr>
    <tr>
      <th>nltk.sent_tokenize</th>
      <td>7752 (3703 + 4049), 58.1s</td>
      <td>1907 (951 + 956), 31.6s</td>
      <td>2212 (829 + 1383), 36.1s</td>
      <td>11390 (6216 + 5174), 42.5s</td>
    </tr>
    <tr>
      <th>segtok.split_single</th>
      <td>8981 (2306 + 6675), 75.2s</td>
      <td>1971 (1090 + 881), 49.5s</td>
      <td>79135 (914 + 78221), 12.9s</td>
      <td>86252 (1206 + 85046), 20.5s</td>
    </tr>
    <tr>
      <th>deepmipt</th>
      <td>4529 (925 + 3604), 35.3s</td>
      <td>579 (114 + 465), 24.5s</td>
      <td>3502 (1647 + 1855), 24.7s</td>
      <td>7487 (5965 + 1522), 26.0s</td>
    </tr>
    <tr>
      <th>razdel.sentenize</th>
      <td>4323 (584 + 3739), 25.7s</td>
      <td>369 (92 + 277), 18.6s</td>
      <td>5872 (1276 + 4596), 18.1s</td>
      <td>4903 (2007 + 2896), 21.7s</td>
    </tr>
  </tbody>
</table>

# Token

In [ ]:
from tqdm import tqdm as log_progress

from razdel.utils import (
    join_path,
    load_lines,
)
from razdel.tests.partition import parse_partitions


lines = log_progress(load_lines(join_path('data', 'syntag_tokens.txt')))
syntag_tokens = list(parse_partitions(lines))

lines = log_progress(load_lines(join_path('data', 'corpora_tokens.txt')))
corpora_tokens = list(parse_partitions(lines))

lines = log_progress(load_lines(join_path('data', 'gicrya_tokens.txt')))
gicrya_tokens = list(parse_partitions(lines))

lines = log_progress(load_lines(join_path('data', 'rnc_tokens.txt')))
rnc_tokens = list(parse_partitions(lines))

In [ ]:
from razdel.eval.zoo import (
    re_tokenize,
    nltk_tokenize,  # see nltk_sentenize
)

# pip install spacy
from razdel.eval.zoo import spacy_tokenize

# pip install git+https://github.com/aatimofeev/spacy_russian_tokenizer.git
from razdel.eval.zoo import spacy_tokenize2

from razdel import tokenize

In [ ]:
%%time
from random import seed, sample

from joblib import Parallel, delayed

from razdel.eval.tests import (
    generate_precision_tests,
    generate_recall_tests,
    correct_precision,
    correct_recall
)
from razdel.eval.report import (
    Dataset,
    Model,
    Metric,
    report_tasks,
    run_task
)


def sample_(items):
    seed(1)
    return sample(items, 10000)


datasets = [
    Dataset('corpora', sample_(corpora_tokens)),
    Dataset('syntag', sample_(syntag_tokens)),
    Dataset('gicrya', sample_(gicrya_tokens)),
    Dataset('rnc', sample_(rnc_tokens)),
]
models = [
    Model('re.findall(\w+|\d+|\p+)', re_tokenize),
    Model('nltk.word_tokenize', nltk_tokenize),
    Model('spacy_tokenize', spacy_tokenize),
    Model('spacy_tokenize2', spacy_tokenize2),
    Model('tokenize', tokenize)
]
metrics = [
    Metric('precision', generate_precision_tests, correct_precision),
    Metric('recall', generate_recall_tests, correct_recall)
]


tasks = list(report_tasks(datasets, models, metrics))
results = Parallel(n_jobs=4)(
    delayed(run_task)(_)
    for _ in log_progress(tasks)
)

In [ ]:
import pandas as pd

from razdel.eval.report import (
    result_rows,
    show_results
)

table = pd.DataFrame(result_rows(results))
print(show_results(table).to_html())
show_results(table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>corpora</th>
      <th>syntag</th>
      <th>gicrya</th>
      <th>rnc</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>re.findall(\w+|\d+|\p+)</th>
      <td>1863 (1857 + 6), 13.2s</td>
      <td>1613 (1460 + 153), 13.5s</td>
      <td>1188 (1188 + 0), 10.9s</td>
      <td>5005 (5002 + 3), 11.0s</td>
    </tr>
    <tr>
      <th>nltk.word_tokenize</th>
      <td>1666 (120 + 1546), 136.3s</td>
      <td>1685 (313 + 1372), 138.6s</td>
      <td>169 (169 + 0), 101.3s</td>
      <td>1987 (1752 + 235), 111.3s</td>
    </tr>
    <tr>
      <th>spacy_tokenize</th>
      <td>2846 (1122 + 1724), 72.0s</td>
      <td>2068 (987 + 1081), 71.7s</td>
      <td>905 (905 + 0), 47.6s</td>
      <td>2706 (2706 + 0), 49.9s</td>
    </tr>
    <tr>
      <th>spacy_tokenize2</th>
      <td>2102 (378 + 1724), 85.3s</td>
      <td>1272 (191 + 1081), 84.1s</td>
      <td>226 (226 + 0), 60.0s</td>
      <td>1877 (1877 + 0), 52.7s</td>
    </tr>
    <tr>
      <th>tokenize</th>
      <td>348 (270 + 78), 26.6s</td>
      <td>460 (383 + 77), 25.9s</td>
      <td>151 (151 + 0), 18.6s</td>
      <td>1755 (1752 + 3), 19.4s</td>
    </tr>
  </tbody>
</table>